In [26]:
import time
import numpy as np
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
from pyope.ope import OPE, ValueRange
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import json



In [27]:
# Load keys from JSON file
with open("keys.json", "r") as f:
    key_data = json.load(f)

in_range = ValueRange(0, 2550)
out_range = ValueRange(0, 2**32 - 1)

num_estimators = 10  # or whatever number you are responsible for (e.g., 10 per person)

start_idx = 0  # Change this to 0, 10, 20, 30, or 40 depending on the group member
end_idx = start_idx + num_estimators

ope_keys = [OPE(key.encode(), in_range=in_range, out_range=out_range) 
            for key in key_data["ope_keys"][start_idx:end_idx]]
aes_keys = [bytes.fromhex(k) for k in key_data["aes_keys"][start_idx:end_idx]]


# Print OPE keys and AES keys
print("OPE Keys:")
for idx, key in enumerate(ope_keys):
    print(f"Key {idx + 1}: {key}")

print("\nAES Keys:")
for idx, key in enumerate(aes_keys):
    print(f"Key {idx + 1}: {key.hex()}")

OPE Keys:
Key 1: <pyope.ope.OPE object at 0x000002A675DFCBD0>
Key 2: <pyope.ope.OPE object at 0x000002A675988FD0>
Key 3: <pyope.ope.OPE object at 0x000002A6759E0D90>
Key 4: <pyope.ope.OPE object at 0x000002A698AAB190>
Key 5: <pyope.ope.OPE object at 0x000002A675A58B10>
Key 6: <pyope.ope.OPE object at 0x000002A675F7E3D0>
Key 7: <pyope.ope.OPE object at 0x000002A675F7D150>
Key 8: <pyope.ope.OPE object at 0x000002A675F7C510>
Key 9: <pyope.ope.OPE object at 0x000002A675F7C790>
Key 10: <pyope.ope.OPE object at 0x000002A675F7F110>

AES Keys:
Key 1: 5658deef22c510ddc7de3266c0b8fb986425b724d1d96f5cbe30c94b592a45b4
Key 2: 22ef0cc87fd53ea257ab512c982f4a8be85fabca3c6768312df6e4214049c23f
Key 3: 6017b927f8b9ac84487cf31973867613db0e978f818d515b45a8d936edc92039
Key 4: 9ed0f33d767fa1950de0cacc3932d93ae88c801e3569d396a04c88686d81fcd8
Key 5: 9cb3c1dd8785f3a4fe1e8931862d2f70a922ab8e85ac0e46446d76a2bc438e0b
Key 6: 4ab6759c1f3aeabb075121dfdb4d43f3acefd871b6cc56ac2ceded712494c84c
Key 7: b8098754304a47be74a

Loading Dataset

In [28]:
start_total_time = time.perf_counter()  # Start total execution time

#Load Dataset Used for Testing and Training
start_dataset_load_time = time.perf_counter()

mnist = fetch_openml("mnist_784", version=1, as_frame=False)
X, y = mnist.data.astype("float32"), mnist.target.astype("int")



end_dataset_load_time = time.perf_counter()

dataset_load_time = end_dataset_load_time - start_dataset_load_time
print(f"Dataset Loading Time: {dataset_load_time:.4f} seconds")


Dataset Loading Time: 4.6206 seconds


Scaling Dataset

In [29]:
# Normalize pixel values to [0, 1]
X = X / 255.0

if X.max() <= 1:
    # Rescale dataset from original range to [0, 255]
    X = (X - X.min()) / (X.max() - X.min()) * 255
    X = (X * 10).astype(int)  # Scale to 0–2550
else:
    X = X.astype(int)

Dataset Splitting

In [30]:
# ✅ Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Change the number of samples to be encrypted for testing purposes (can be removed )
num_samples_training = len(X_train)
num_samples_testing = len(X_test)

X_test = X_test[:num_samples_testing]
X_train = X_train[:num_samples_training]

print(f"Number of training samples: {num_samples_training}")
print(f"Number of testing samples: {num_samples_testing}")

Number of training samples: 56000
Number of testing samples: 14000


OPE Initialization and Setup

In [31]:
ope_key = b'some_secure_key'
scale_factor = 10
max_pixel_value = 255 * scale_factor  

#Function to encrypt the dataset using OPE
def encrypt_dataset_with_ope(X, ope_keys):
    encrypted_versions = []

    for idx, ope in enumerate(ope_keys):
        print(f"Encrypting dataset with OPE key {idx + 1}")
        encrypted_X = np.array([
            [ope.encrypt(int(val)) for val in row]
            for row in tqdm(X, desc=f"Encrypting for Tree {idx + 1}")
        ])
        encrypted_versions.append(encrypted_X)

    return encrypted_versions  # List of arrays, one per tree


Testing Data Encryption using OPE

In [32]:
def load_or_encrypt_dataset(X, ope_keys):
    encrypted_versions = []
    encryption_times = []

    for idx, ope in enumerate(ope_keys):
        key_name = key_data["ope_keys"][idx]  # raw string from keys.json
        encrypted_file_path = f"X_test_encrypted_{key_name}.npy"

        if os.path.exists(encrypted_file_path):
            encrypted_X = np.load(encrypted_file_path)

            # ✅ Check if cached file has the correct number of samples
            if encrypted_X.shape[0] != X.shape[0]:
                print(f"❌ Cached file for tree {idx + 1} has incorrect shape: {encrypted_X.shape}, expected {X.shape}. Re-encrypting...")
                os.remove(encrypted_file_path)
                
                # Force re-encryption below
                print(f"🔁 Encrypting dataset for tree {idx + 1}...")
                start = time.perf_counter()
                encrypted_X = np.array([
                    [ope.encrypt(int(val)) for val in row]
                    for row in tqdm(X, desc=f"Encrypting for Tree {idx + 1}")
                ])
                end = time.perf_counter()

                np.save(encrypted_file_path, encrypted_X)
                print(f"✅ Saved encrypted data: {encrypted_file_path}")
                encryption_times.append(end - start)
            else:
                print(f"✅ Loaded cached encrypted data for tree {idx + 1}")
                encryption_times.append(0)

        else:
            print(f"🔐 Encrypting dataset for tree {idx + 1}...")
            start = time.perf_counter()
            encrypted_X = np.array([
                [ope.encrypt(int(val)) for val in row]
                for row in tqdm(X, desc=f"Encrypting for Tree {idx + 1}")
            ])
            end = time.perf_counter()

            np.save(encrypted_file_path, encrypted_X)
            print(f"✅ Saved encrypted data: {encrypted_file_path}")
            encryption_times.append(end - start)

        encrypted_versions.append(encrypted_X)

    return encrypted_versions, encryption_times


In [34]:
import os

# Ensure the folder exists
os.makedirs("Encrypted_Dataset", exist_ok=True)

# Encrypt test data using per-tree OPE keys
start_test_data_encryption_time = time.perf_counter()

print(f"X_test size: {X_test.shape}")

X_test_encrypted_per_tree = []
encryption_times = []

for idx, ope in enumerate(ope_keys):
    key_name = key_data["ope_keys"][idx]  # Use the raw key string
    file_path = f"Encrypted_Dataset/X_test_encrypted_{key_name}.npy"

    if os.path.exists(file_path):
        print(f"Loaded cached encrypted data for tree {idx + 1}")
        encrypted_X = np.load(file_path)
        encryption_times.append(0)
    else:
        print(f"Encrypting test data for tree {idx + 1} (key: {key_name[:8]}...)")
        start = time.perf_counter()
        encrypted_X = np.array([
            [ope.encrypt(int(val)) for val in row]
            for row in tqdm(X_test, desc=f"Encrypting for Tree {idx + 1}")
        ])
        end = time.perf_counter()
        np.save(file_path, encrypted_X)
        print(f"Saved encrypted test data to {file_path}")
        encryption_times.append(end - start)

    X_test_encrypted_per_tree.append(encrypted_X)

end_test_data_encryption_time = time.perf_counter()

# Fallback in case all were loaded
if all(t == 0 for t in encryption_times):
    test_data_encryption_time = end_test_data_encryption_time - start_test_data_encryption_time
else:
    test_data_encryption_time = sum(encryption_times)

print(f"Size of X_test_encrypted_per_tree: {len(X_test_encrypted_per_tree)} trees")
for idx, encrypted_tree in enumerate(X_test_encrypted_per_tree):
    print(f"Tree {idx + 1} encrypted data size: {encrypted_tree.shape}")

print(f"Total Dataset Encryption Time (Multi-Key): {test_data_encryption_time:.4f} seconds")


X_test size: (14000, 784)
Loaded cached encrypted data for tree 1
Loaded cached encrypted data for tree 2
Loaded cached encrypted data for tree 3
Loaded cached encrypted data for tree 4
Loaded cached encrypted data for tree 5
Loaded cached encrypted data for tree 6
Loaded cached encrypted data for tree 7
Loaded cached encrypted data for tree 8
Loaded cached encrypted data for tree 9
Loaded cached encrypted data for tree 10
Size of X_test_encrypted_per_tree: 10 trees
Tree 1 encrypted data size: (14000, 784)
Tree 2 encrypted data size: (14000, 784)
Tree 3 encrypted data size: (14000, 784)
Tree 4 encrypted data size: (14000, 784)
Tree 5 encrypted data size: (14000, 784)
Tree 6 encrypted data size: (14000, 784)
Tree 7 encrypted data size: (14000, 784)
Tree 8 encrypted data size: (14000, 784)
Tree 9 encrypted data size: (14000, 784)
Tree 10 encrypted data size: (14000, 784)
Total Dataset Encryption Time (Multi-Key): 0.9654 seconds


Random Forest Initialization & Training

In [35]:
start_training_time = time.perf_counter()

clf_ope = RandomForestClassifier(n_estimators=num_estimators, max_depth=20, random_state=42, min_samples_split=2)
clf_ope.fit(X_train, y_train)

end_training_time = time.perf_counter()
training_time = end_training_time - start_training_time

print(f"Random Forest Training Time: {training_time:.4f} seconds")

Random Forest Training Time: 3.7848 seconds


AES Function Definition and Label Encryption (Using ECB as the mode of operation)

In [36]:
# ✅ AES Encrypt Function
def aes_encrypt(data, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    padded_data = data.ljust(16)
    ciphertext = encryptor.update(padded_data.encode()) + encryptor.finalize()
    return ciphertext

# ✅ AES Decrypt Function
def aes_decrypt(ciphertext, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext).decode().strip()

# ⏱️ Encrypt leaf labels per tree using that tree's AES key
import base64

start_label_encryption_time = time.perf_counter()

encrypted_leaf_values = []
for idx, tree in enumerate(clf_ope.estimators_):
    tree_values = []
    for val in tree.tree_.value[:, 0, 0]:
        ciphertext = aes_encrypt(str(int(val)), aes_keys[idx])
        encrypted_value = base64.b64encode(ciphertext).decode()
        tree_values.append(encrypted_value)
    encrypted_leaf_values.append(tree_values)

end_label_encryption_time = time.perf_counter()
label_encryption_time = end_label_encryption_time - start_label_encryption_time

print(f"✅ Encrypted Leaf Values for {len(encrypted_leaf_values)} trees")
print(f"🔐 AES Label Encryption Time: {label_encryption_time:.4f} seconds")


✅ Encrypted Leaf Values for 10 trees
🔐 AES Label Encryption Time: 1.5760 seconds


Thresholds Encryption

In [37]:
# Make sure the directory exists
os.makedirs("Encrypted_Thresholds", exist_ok=True)

start_threshold_encryption_time = time.perf_counter()
encrypted_thresholds = []

for idx, (tree, ope) in enumerate(zip(clf_ope.estimators_, ope_keys)):
    key_name = key_data["ope_keys"][idx]  # Get key name from JSON
    file_path = f"Encrypted_Thresholds/encrypted_thresholds_{key_name}.npy"

    if os.path.exists(file_path):
        thresholds = np.load(file_path, allow_pickle=True)
        encrypted_thresholds.append(thresholds.tolist())
        print(f"Loaded cached thresholds for key {key_name}")
    else:
        print(f"Encrypting thresholds for key {key_name}")
        tree_thresholds = []
        for th in tree.tree_.threshold:
            if th != -2:
                tree_thresholds.append(ope.encrypt(int(th)))
            else:
                tree_thresholds.append(None)
        encrypted_thresholds.append(tree_thresholds)
        np.save(file_path, tree_thresholds)
        print(f"Saved thresholds to {file_path}")

end_threshold_encryption_time = time.perf_counter()
threshold_encryption_time = end_threshold_encryption_time - start_threshold_encryption_time

print(f"Encrypted Thresholds for {len(encrypted_thresholds)} trees")
print(f"Threshold Encryption Time: {threshold_encryption_time:.4f} seconds")

Loaded cached thresholds for key 5f5e8a3fe2ba8ef9979983ebad66c7d7
Loaded cached thresholds for key 456f782dbde337bd96fb673941f9335a
Loaded cached thresholds for key 216770f64070844fddf2eadd8aa7b100
Loaded cached thresholds for key 6ce2504f1c23915e0062045b639b2c7f
Loaded cached thresholds for key c456d38cab06f85268dfa40620da8eee
Loaded cached thresholds for key e5af9c5870aeb77a3dd360340d768569
Loaded cached thresholds for key f161e54ba25699c70be43cb547b1b198
Loaded cached thresholds for key b29671a1bab218eb7d364a3831cc2b82
Loaded cached thresholds for key fcc93a3653c266ef4896d2c82e45310a
Loaded cached thresholds for key faff2ba978176f936b83c6c040388a62
Encrypted Thresholds for 10 trees
Threshold Encryption Time: 0.0142 seconds


Leaf Node Encryption

In [38]:
# Encrypt Leaf Node Labels Per Tree Using AES Keys

start_leaf_encryption_time = time.perf_counter()

encrypted_leaf_values = []

for i, tree in enumerate(clf_ope.estimators_):
    current_key = aes_keys[i]
    tree_leaf_map = {}
    for node in range(tree.tree_.node_count):
        if tree.tree_.feature[node] == -2:  # It's a leaf node
            label = str(tree.tree_.value[node].argmax())  # or int(...) if needed
            tree_leaf_map[node] = aes_encrypt(label, current_key)
    encrypted_leaf_values.append(tree_leaf_map)

end_leaf_encryption_time = time.perf_counter()
leaf_encryption_time = end_leaf_encryption_time - start_leaf_encryption_time

print(f"✅ Encrypted leaf node labels for {len(encrypted_leaf_values)} trees")
print(f"🔐 Leaf Node Encryption Time: {leaf_encryption_time:.4f} seconds")


✅ Encrypted leaf node labels for 10 trees
🔐 Leaf Node Encryption Time: 0.9258 seconds


Dataset Encryption Functions

In [39]:
# Function to Encrypt One Image with a Given OPE Key
def encrypt_image(image, ope_key):
    return [ope_key.encrypt(int(pixel)) for pixel in image]

# Function to Encrypt Entire Dataset Separately Per OPE Key (Multi-Key)
def encrypt_dataset_with_ope(X, ope_keys):
    encrypted_versions = []

    for idx, ope_key in enumerate(ope_keys):
        print(f"\n🔐 Encrypting dataset with OPE key {idx + 1}")
        encrypted_X = []
        for i, image in enumerate(tqdm(X, desc=f"Encrypting for Tree {idx + 1}"), start=1):
            start_time = time.time()
            encrypted_image = encrypt_image(image, ope_key)
            encryption_time = time.time() - start_time
            print(f"{i}: Image Encryption Time: {encryption_time:.4f} sec")
            encrypted_X.append(encrypted_image)

        print(f"✅ Number of encrypted images for tree {idx + 1}: {len(encrypted_X)}")
        encrypted_versions.append(np.array(encrypted_X))

    return encrypted_versions  # List of datasets, one per tree


In [40]:
def simulate_pre(ciphertext, original_key, target_key):
    plaintext = aes_decrypt(ciphertext, original_key)
    return aes_encrypt(plaintext, target_key)


In [41]:
def majority_vote_with_pre(encrypted_votes, aes_keys, global_key):
    """
    Applies simulated PRE to encrypted votes from different trees,
    then performs majority voting on re-encrypted labels.
    """
    reencrypted_votes = [
        simulate_pre(ciphertext, original_key, global_key)
        for ciphertext, original_key in zip(encrypted_votes, aes_keys)
    ]

    # Decrypt using the global key
    decrypted_votes = [int(aes_decrypt(c, global_key)) for c in reencrypted_votes]
    return Counter(decrypted_votes).most_common(1)[0][0]


In [42]:
def secure_classify(model, encrypted_X_per_tree, encrypted_thresholds, encrypted_leaf_values, aes_keys, global_key):
    """
    Secure classification that collects encrypted votes and delegates voting to majority_vote_with_pre.
    """
    encrypted_votes = []

    for tree_idx, tree in enumerate(model.estimators_):
        node = 0
        tree_thresholds = encrypted_thresholds[tree_idx]
        encrypted_X = encrypted_X_per_tree[tree_idx]
        tree_key = aes_keys[tree_idx]

        while tree.tree_.feature[node] != -2:
            feature_idx = tree.tree_.feature[node]
            encrypted_threshold = tree_thresholds[node]

            if encrypted_X[feature_idx] < encrypted_threshold:
                node = tree.tree_.children_left[node]
            else:
                node = tree.tree_.children_right[node]

        encrypted_label = encrypted_leaf_values[tree_idx][node]
        encrypted_votes.append(encrypted_label)

    return majority_vote_with_pre(encrypted_votes, aes_keys, global_key)


In [43]:
global_aes_key = os.urandom(16)  # Or use get_random_bytes(16) from Crypto

def secure_classify_dataset(model, X_encrypted_per_tree, encrypted_thresholds, encrypted_leaf_values, aes_keys, global_key):
    num_samples = len(X_encrypted_per_tree[0])  # number of test samples
    predictions = []

    for sample_idx in tqdm(range(num_samples), desc="Classifying Encrypted Test Samples"):
        # Get this sample's encrypted version from all trees
        sample_per_tree = [X_encrypted_per_tree[tree_idx][sample_idx] for tree_idx in range(len(model.estimators_))]

        pred = secure_classify(
            model=model,
            encrypted_X_per_tree=sample_per_tree,
            encrypted_thresholds=encrypted_thresholds,
            encrypted_leaf_values=encrypted_leaf_values,
            aes_keys=aes_keys,
            global_key=global_key
        )
        predictions.append(pred)

    return np.array(predictions)



In [44]:
# Print the size of X_test_encrypted_per_tree
print(f"Size of X_test_encrypted_per_tree: {len(X_test_encrypted_per_tree)} trees")
for idx, encrypted_tree in enumerate(X_test_encrypted_per_tree):
    print(f"Tree {idx + 1} encrypted data size: {encrypted_tree.shape}")
    print(encrypted_tree[0])  # Just the first image sample
    print(f"Length of images {len(encrypted_tree[0])}")  # Just the first image sample


Size of X_test_encrypted_per_tree: 10 trees
Tree 1 encrypted data size: (14000, 784)
[    230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496     230496     230496
     230496     230496     230496     230496

In [45]:
for i, tree_data in enumerate(X_test_encrypted_per_tree):
    print(f"Tree {i + 1} encrypted data shape: {tree_data.shape}")


Tree 1 encrypted data shape: (14000, 784)
Tree 2 encrypted data shape: (14000, 784)
Tree 3 encrypted data shape: (14000, 784)
Tree 4 encrypted data shape: (14000, 784)
Tree 5 encrypted data shape: (14000, 784)
Tree 6 encrypted data shape: (14000, 784)
Tree 7 encrypted data shape: (14000, 784)
Tree 8 encrypted data shape: (14000, 784)
Tree 9 encrypted data shape: (14000, 784)
Tree 10 encrypted data shape: (14000, 784)


In [46]:

# ✅ Measure time taken for classification using multi-key + PRE
start_time = time.time()
print("🔐 Performing Secure Classification with PRE...")

y_pred_encrypted = secure_classify_dataset(
    model=clf_ope,
    X_encrypted_per_tree=X_test_encrypted_per_tree,
    encrypted_thresholds=encrypted_thresholds,
    encrypted_leaf_values=encrypted_leaf_values,
    aes_keys=aes_keys,
    global_key=global_aes_key
)

print("Predictions:", y_pred_encrypted)
print("True Labels:", y_test[:num_samples_testing])

classification_time = time.time() - start_time
print(f"✅ Secure Classification Time: {classification_time:.4f} seconds")


🔐 Performing Secure Classification with PRE...


Classifying Encrypted Test Samples: 100%|██████████| 14000/14000 [00:11<00:00, 1266.58it/s]

Predictions: [8 4 8 ... 2 7 1]
True Labels: [8 4 8 ... 2 7 1]
✅ Secure Classification Time: 11.0636 seconds


In [47]:
# ✅ Sanity check: make sure prediction length matches test labels
print("Length of predictions:", len(y_pred_encrypted))
print("Length of ground truth:", len(y_test[:num_samples_testing]))

if len(y_pred_encrypted) == len(y_test[:num_samples_testing]):
    # ✅ Compute accuracy
    secure_accuracy = accuracy_score(y_test[:num_samples_testing], y_pred_encrypted)
    print(f"✅ Secure Random Forest Accuracy on Encrypted Dataset: {secure_accuracy:.4f}")
else:
    print("❌ Error: Prediction length does not match test labels. Cannot compute accuracy.")


Length of predictions: 14000
Length of ground truth: 14000
✅ Secure Random Forest Accuracy on Encrypted Dataset: 0.9442


In [48]:
# Ensure all timer variables are defined to avoid NameError
if 'start_threshold_encryption_time' not in globals():
    start_threshold_encryption_time = end_threshold_encryption_time = time.perf_counter()

if 'start_label_encryption_time' not in globals():
    start_label_encryption_time = end_label_encryption_time = time.perf_counter()

if 'start_dataset_encryption_time' not in globals():
    start_dataset_encryption_time = end_dataset_encryption_time = time.perf_counter()

if 'start_rf_training_time' not in globals():
    start_rf_training_time = end_rf_training_time = time.perf_counter()

end_total_time = time.perf_counter()  # End total execution time

total_time = end_total_time - start_total_time
dataset_encryption_time = end_dataset_encryption_time - start_dataset_encryption_time
rf_training_time = end_rf_training_time - start_rf_training_time
threshold_encryption_time = end_threshold_encryption_time - start_threshold_encryption_time


# Recompute actual total from all components to ensure percentages are meaningful
effective_total_time = (
    dataset_load_time +
    test_data_encryption_time +
    training_time +
    threshold_encryption_time +
    classification_time
)

dataset_load_percentage = (dataset_load_time / effective_total_time) * 100
test_data_encryption_percentage = (test_data_encryption_time / effective_total_time) * 100
rf_training_percentage = (training_time / effective_total_time) * 100
threshold_encryption_percentage = (threshold_encryption_time / effective_total_time) * 100
classification_percentage = (classification_time / effective_total_time) * 100


# Combine encryption and classification times
total_throughput_time = test_data_encryption_time + classification_time

print("\n===== Execution Time Summary =====")
print(f"Total Execution Time: {effective_total_time:.4f} seconds")
print(f"Dataset Load Time: {dataset_load_time:.4f} seconds ({dataset_load_percentage:.2f}%)")
print(f"Test Data Encryption Time: {test_data_encryption_time:.4f} seconds ({test_data_encryption_percentage:.2f}%)")
print(f"Random Forest Training Time: {training_time:.4f} seconds ({rf_training_percentage:.2f}%)")
print(f"Threshold Encryption Time: {threshold_encryption_time:.4f} seconds ({threshold_encryption_percentage:.2f}%)")
print(f"Secure Classification Time: {classification_time:.4f} seconds ({classification_percentage:.2f}%)")

print("\n===== Secure Classification Results =====")
print(f"Secure Random Forest Accuracy on Encrypted MNIST: {secure_accuracy:.4f}")
print(f"Number of Decision Trees (num_estimators): {num_estimators}")
print(f"Number of Images Used for Training: {len(X_train)}")
print(f"Number of Images Used for Testing: {len(X_test)}")

encryption_percentage_throughput = (test_data_encryption_time / total_throughput_time) * 100
classification_percentage_througput = (classification_time / total_throughput_time) * 100

print("\n===== Throughput =====")
throughput = len(X_test) / total_throughput_time
print(f"Total Throughput Time: {total_throughput_time:.4f} seconds")
print(f"Throughput: {throughput:.2f} samples/second")
print(f"Percentage of Test Data Encryption Time vs Throughput: {encryption_percentage_throughput:.2f}%")
print(f"Percentage of Classification Time vs Throughput: {classification_percentage_througput:.2f}%")



===== Execution Time Summary =====
Total Execution Time: 20.4485 seconds
Dataset Load Time: 4.6206 seconds (22.60%)
Test Data Encryption Time: 0.9654 seconds (4.72%)
Random Forest Training Time: 3.7848 seconds (18.51%)
Threshold Encryption Time: 0.0142 seconds (0.07%)
Secure Classification Time: 11.0636 seconds (54.10%)

===== Secure Classification Results =====
Secure Random Forest Accuracy on Encrypted MNIST: 0.9442
Number of Decision Trees (num_estimators): 10
Number of Images Used for Training: 56000
Number of Images Used for Testing: 14000

===== Throughput =====
Total Throughput Time: 12.0290 seconds
Throughput: 1163.85 samples/second
Percentage of Test Data Encryption Time vs Throughput: 8.03%
Percentage of Classification Time vs Throughput: 91.97%
